In [24]:
import json

import pandas as pd

from pandas.io.json import json_normalize
from sqlalchemy import create_engine

In [25]:
with open('data/data.json') as jsonfile:
    data = json.load(jsonfile)['data']

In [26]:
data[:2]

[{'id': '5b3e1972d6342b00010020de',
  'location': {'standard': 'http://s3.amazonaws.com/ndnp-jpeg-surrogates/iune_edition_ver02/data/sn92053934/00295872640/1917050701/0084.jpg'},
  'region': {'toolName': 'rectangleTool',
   'color': '#fe0000',
   'x': 810.293696532371,
   'y': 318.0028157678859,
   'width': 838.3594507992422,
   'height': 2522.340515977095,
   'label': 'Illustration/Photo'},
  'width': 5912,
  'height': 7667,
  'meta_data': {'set_key': 'http://chroniclingamerica.loc.gov/lccn/sn92053934/1917-05-07/ed-1/seq-7/',
   'subject_url': 'http://chroniclingamerica.loc.gov/lccn/sn92053934/1917-05-07/ed-1/seq-7/',
   'subject_description': 'Rock Island Argus. May 07, 1917. Page 7',
   'resize': '1.000000',
   'alto': 'http://s3.amazonaws.com/ndnp-batches/iune_edition_ver02/data/sn92053934/00295872640/1917050701/0084.xml'},
  'data': {'caption': 'NOTED BRITISHER WITH COMMISSION\nMaj. Gen. G. T. M. Bridges\nMaj. Gen. G. T. M. Bridges, chief military representative of the British com

In [27]:
# the json data contains has inconsistencies: sometimes 
# the 'data' attribute is at root level, somtimes it's
# nested in a single-item list, under a 'values' attribute

# move 'data' attribute to root if necessary
for i, doc in enumerate(data):
    values_attrib = doc['data'].get('values')
    if values_attrib: 
        data[i]['data'] = values_attrib[0]

In [28]:
# flatten the data, create dataframe
df = json_normalize(data)
# json_normalize uses periods in col names; change to underscore
df.columns = df.columns.map(lambda x: x.replace('.', '_'))

df.head(3)

,data_caption,data_category,data_creator,height,id,location_standard,meta_data_alto,meta_data_resize,meta_data_set_key,meta_data_subject_description,meta_data_subject_url,region_color,region_height,region_label,region_toolName,region_width,region_x,region_y,width
0,NOTED BRITISHER WITH COMMISSION\nMaj. Gen. G. ...,Photograph,HARRIS & EWING,7667,5b3e1972d6342b00010020de,http://s3.amazonaws.com/ndnp-jpeg-surrogates/i...,http://s3.amazonaws.com/ndnp-batches/iune_edit...,1.000000,http://chroniclingamerica.loc.gov/lccn/sn92053...,"Rock Island Argus. May 07, 1917. Page 7",http://chroniclingamerica.loc.gov/lccn/sn92053...,#fe0000,2522.340516,Illustration/Photo,rectangleTool,838.359451,810.293697,318.002816,5912
1,Not That She Didn't Try\nHOW DO YOU LIKE MY HA...,Comics/Cartoon,C.A. VOIGHT,6451,5b21b9cdd6342b00010018a4,http://s3.amazonaws.com/ndnp-jpeg-surrogates/p...,http://s3.amazonaws.com/ndnp-batches/pst_davey...,1.000000,http://chroniclingamerica.loc.gov/lccn/sn83045...,"Evening public ledger. March 04, 1918. Page 13",http://chroniclingamerica.loc.gov/lccn/sn83045...,#fe0000,1020.800308,Illustration/Photo,rectangleTool,3646.961825,659.593274,261.743669,5104
2,"HEAD OF THE VETERANS\nGen. George P. Harrison,...",Photograph,NaN,9168,5b312a24d6342b0001001d7a,http://s3.amazonaws.com/ndnp-jpeg-surrogates/f...,http://s3.amazonaws.com/ndnp-batches/fu_criser...,1.000000,http://chroniclingamerica.loc.gov/lccn/sn84027...,"The Ocala evening star. September 17, 1918. Pa...",http://chroniclingamerica.loc.gov/lccn/sn84027...,#fe0000,1613.238984,Illustration/Photo,rectangleTool,944.593873,2281.859784,7270.189167,6368


In [29]:
# drop columns we probably won't need
drop_cols = [
    'meta_data_resize',
    'region_color',
    'region_toolName',
    'region_label',
]
for col in drop_cols:
    df.drop(col, axis=1, inplace=True)

In [30]:
# show data types that pandas has inferred
df.dtypes

data_caption                      object
data_category                     object
data_creator                      object
height                             int64
id                                object
location_standard                 object
meta_data_alto                    object
meta_data_set_key                 object
meta_data_subject_description     object
meta_data_subject_url             object
region_height                    float64
region_width                     float64
region_x                         float64
region_y                         float64
width                              int64
dtype: object

In [31]:
df.count()

data_caption                     1459
data_category                    1460
data_creator                      422
height                           1474
id                               1474
location_standard                1474
meta_data_alto                   1474
meta_data_set_key                1474
meta_data_subject_description    1474
meta_data_subject_url            1474
region_height                    1474
region_width                     1474
region_x                         1474
region_y                         1474
width                            1474
dtype: int64

In [32]:
# convert NaNs to blanks in text columns that have missing columns
na_blank_cols = ['data_caption', 'data_category', 'data_creator']
for col in na_blank_cols:
    df[col] = df[col].fillna('')


In [61]:
# add a column that is the hash of the image url,
# we'll use this as an image identifier

import hashlib

# func to return hash for a string
def hashstr(s):
    return hashlib.md5(s.encode('utf8')).hexdigest()

# apply to new column
df['img_url_hash'] = df.apply(lambda r: hashstr(r['location_standard']), axis = 1)

In [62]:
# save data as sqlite db
eng = create_engine('sqlite:///db/data.sqlite3', echo=False)
df.to_sql('data', eng, index=False, if_exists='replace')

In [67]:
# re-read data from sql and check data types
pd.read_sql("select * from data", eng).dtypes

data_caption                      object
data_category                     object
data_creator                      object
height                             int64
id                                object
location_standard                 object
meta_data_alto                    object
meta_data_set_key                 object
meta_data_subject_description     object
meta_data_subject_url             object
region_height                    float64
region_width                     float64
region_x                         float64
region_y                         float64
width                              int64
img_url_hash                      object
dtype: object

In [68]:
# re-read data from sql and check data types
pd.read_sql("select * from data", eng).count()

data_caption                     1474
data_category                    1474
data_creator                     1474
height                           1474
id                               1474
location_standard                1474
meta_data_alto                   1474
meta_data_set_key                1474
meta_data_subject_description    1474
meta_data_subject_url            1474
region_height                    1474
region_width                     1474
region_x                         1474
region_y                         1474
width                            1474
img_url_hash                     1474
dtype: int64

In [85]:
# download image files

import requests
import os.path

sql = "select location_standard, img_url_hash from data"
result = eng.execute(sql)
for row in result:
    uhash = row[1]
    url = row[0]
    # image filename is based on url hash
    path = 'images/{}.jpg'.format(uhash)
    # skip if we've already downloaded this file
    if os.path.exists(path):
        continue
    # otherwise, download and save it
    else:
        r = requests.get(url, stream=True)
        # if http error, skip and print error
        if not r.status_code == 200:
            print('{} error for {}'.format(r.status_code, url))
            continue
        else:
            with open(path, 'wb') as f:
                for chunk in r:
                    f.write(chunk)
